In [ ]:
pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.1 MB/s eta 0:00:00


In [ ]:
pip install transformers

In [ ]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.5/383.5 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
ds = load_dataset("codeparrot/codeparrot-valid-more-filtering", streaming=True, split="train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/999 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


In [ ]:
import subprocess
import sys
import io
import contextlib
import importlib
import os
import shutil

def python_sandbox(code):
    packages = ["pandas", "matplotlib", "seaborn", "requests", "numpy", "IPython"]
    sandbox_dir = "/content/sandbox_files"

    def install_package(package):
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

    def create_sandbox_directory():
        if not os.path.exists(sandbox_dir):
            os.makedirs(sandbox_dir)

    def get_safe_modules():
        safe_modules = {
            '__builtins__': __builtins__,
            'os': os,
        }
        for package in packages:
            safe_modules[package] = importlib.import_module(package)
        return safe_modules

    # Install packages and create sandbox directory
    for package in packages:
        try:
            __import__(package)
        except ImportError:
            install_package(package)
    create_sandbox_directory()

    # Capture the stdout and stderr
    output = io.StringIO()
    error = io.StringIO()

    original_cwd = os.getcwd()
    try:
        os.chdir(sandbox_dir)
        with contextlib.redirect_stdout(output), contextlib.redirect_stderr(error):
            exec(code, get_safe_modules())

        # Check if any files were created in the sandbox directory
        created_files = os.listdir(sandbox_dir)
        if created_files:
            output_str = output.getvalue()
            output_str += f"\nCreated files: {', '.join(created_files)}"
            for file in created_files:
                file_path = os.path.join(sandbox_dir, file)
                output_str += f"\nFile '{file}' contents:\n"
                if file.endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                    output_str += "[Binary image file]"
                else:
                    try:
                        with open(file_path, 'r') as f:
                            output_str += f.read()
                    except UnicodeDecodeError:
                        output_str += "[Binary file]"

            return output_str, error.getvalue()
        else:
            return output.getvalue(), error.getvalue()

    except Exception as e:
        return "", str(e)
    finally:
        os.chdir(original_cwd)

In [ ]:
import openai
import pandas as pd
from transformers import GPT2Tokenizer
import json
import time
import re
import os

# Initialize tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Initialize OpenAI API client
openai.api_key = ""  # Replace with your actual API key

# Assuming ds is your dataset with 'content' being the code samples
# Extract code samples that have more than 150 tokens
filtered_code_samples = []
print("Filtering code samples based on token count...")

for idx, sample in enumerate(ds):  # ds should be defined earlier in your script or imported
    token_count = len(tokenizer.encode(sample['content']))
    if token_count < 1500:
        filtered_code_samples.append(sample['content'])
    if len(filtered_code_samples) >= 1000:
        print("Collected 1000 samples. Stopping.")
        break

print("Finished filtering code samples.")

# Create a DataFrame with the 'content' column
df_filtered = pd.DataFrame(filtered_code_samples, columns=["content"])

# Now, we will use the OpenAI Batch API to perform the initial filtering using evaluate_code_sandbox_compatibility function

# First, define the code for the sandbox
sandbox_code = '''
def python_sandbox(code):
    packages = ["pandas", "matplotlib", "seaborn", "requests", "numpy", "IPython"]
    sandbox_dir = "/content/sandbox_files"

    def install_package(package):
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

    def create_sandbox_directory():
        if not os.path.exists(sandbox_dir):
            os.makedirs(sandbox_dir)

    def get_safe_modules():
        safe_modules = {
            '__builtins__': __builtins__,
            'os': os,
        }
        for package in packages:
            safe_modules[package] = importlib.import_module(package)
        return safe_modules

    # Install packages and create sandbox directory
    for package in packages:
        try:
            __import__(package)
        except ImportError:
            install_package(package)
    create_sandbox_directory()

    # Capture the stdout and stderr
    output = io.StringIO()
    error = io.StringIO()

    original_cwd = os.getcwd()
    try:
        os.chdir(sandbox_dir)
        with contextlib.redirect_stdout(output), contextlib.redirect_stderr(error):
            exec(code, get_safe_modules())

        # Check if any files were created in the sandbox directory
        created_files = os.listdir(sandbox_dir)
        if created_files:
            output_str = output.getvalue()
            output_str += f"\\nCreated files: {', '.join(created_files)}"
            for file in created_files:
                file_path = os.path.join(sandbox_dir, file)
                output_str += f"\\nFile '{file}' contents:\\n"
                if file.endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                    output_str += "[Binary image file]"
                else:
                    try:
                        with open(file_path, 'r') as f:
                            output_str += f.read()
                        except UnicodeDecodeError:
                            output_str += "[Binary file]"

            return output_str, error.getvalue()
        else:
            return output.getvalue(), error.getvalue()

    except Exception as e:
        return "", str(e)
    finally:
        os.chdir(original_cwd)
'''

# Modify evaluate_code_sandbox_compatibility function to include the sandbox code in the prompt
def create_evaluate_prompt(code):
    prompt = f"""
You are tasked with determining whether a given piece of Python code can reasonably be modified to run successfully in a Python sandbox environment. The sandbox has specific constraints and functionalities, which you must keep in mind while evaluating the code. Your task is to reason about the code's compatibility with the sandbox environment and decide if it is possible to modify the code in a way that would make it runnable within this sandbox. If it is possible, you should answer 'Yes.' If it is not possible under reasonable modifications, answer 'No.'


### Sandbox Constraints to Consider:
- The sandbox allows the following Python packages: `pandas`, `matplotlib`, `seaborn`, `requests`, `numpy`, `IPython`.
- The code is executed in a restricted environment where only a subset of safe modules and packages is accessible.
- File creation is possible in the sandbox directory, but special handling is needed for binary files (like images) or non-text files.
- Complex, system-level operations or any packages not included in the list above are not supported in the sandbox.
- Any risky operations (e.g., networking not using `requests`, system manipulations, subprocess calls) may not be executed safely or at all.

### Special Consideration for External Dependencies:
- If the code depends on external libraries, modules, or APIs that are not standard or not among the permitted packages, evaluate whether these dependencies can be easily simulated or replaced without seeing the original definitions.
- If the code relies heavily on complex or custom modules that are not self-explanatory and cannot be easily reproduced in the sandbox without the original function definitions, consider this code as incompatible.

### Your Evaluation:
1. **Code Analysis:** Carefully analyze the provided code, identifying any components that could cause compatibility issues with the sandbox, especially external dependencies.
2. **Possible Modifications:** Consider if the code can be modified within reason to adhere to the sandbox's constraints. If so, briefly outline the possible changes.
3. **Reasoning:** Explain your reasoning, considering all constraints, external dependencies, and potential modifications.
4. **Verdict:** Conclude with a final verdict on whether the code can be made compatible with the sandbox by placing 'Yes' or 'No' between `<verdict>` and `</verdict>` tags.

### Here’s the code to evaluate:

{code}
"""
    return prompt

# Now, we need to create batch tasks for the OpenAI Batch API
# We will prepare a list of tasks in JSONL format

tasks = []

for index, row in df_filtered.iterrows():
    code = row['content']
    prompt = create_evaluate_prompt(code)
    task = {
        "custom_id": f"task-{index}",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": "gpt-4o-mini",
            "temperature": 0.7,
            "messages": [
                {
                    "role": "user",
                    "content": prompt
                }
            ]
        }
    }
    tasks.append(task)

# Save tasks to a JSONL file
tasks_file_name = "batch_tasks_evaluate_code.jsonl"

with open(tasks_file_name, 'w') as file:
    for task in tasks:
        json.dump(task, file)
        file.write('\n')

# Now, upload the tasks file to OpenAI and create a batch job
batch_file = client.files.create(
    file=open(tasks_file_name, "rb"),
    purpose="batch"
)
batch_input_file_id = batch_file.id

batch_job = client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

print("batch created")
# Now, we can periodically check the status of the batch job
batch_job_id = batch_job.id
batch_status = openai.batches.retrieve(batch_job_id)
print(batch_status)




Filtering code samples based on token count...


Token indices sequence length is longer than the specified maximum sequence length for this model (1185 > 1024). Running this sequence through the model will result in indexing errors


Collected 1000 samples. Stopping.
Finished filtering code samples.
batch created
Batch(id='batch_66fdba8d74e081909c9ba0d5e89f42ec', completion_window='24h', created_at=1727904397, endpoint='/v1/chat/completions', input_file_id='file-Hb2ynHvSbWG7onMdFjjcioEO', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1727990797, failed_at=None, finalizing_at=None, in_progress_at=None, metadata=None, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))
Batch job status: Batch(id='batch_66fdba8d74e081909c9ba0d5e89f42ec', completion_window='24h', created_at=1727904397, endpoint='/v1/chat/completions', input_file_id='file-Hb2ynHvSbWG7onMdFjjcioEO', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1727990797, failed_at=None, finalizing_at=None, in_progress

AttributeError: module 'openai' has no attribute 'Batch'

In [ ]:
# Wait for the batch job to complete
while True:
    if(openai.batches.retrieve(batch_job_id) == 'completed'):
      break
    print(f"Batch job status: {openai.batches.retrieve(batch_job_id)}")
    time.sleep(60)  # Wait for 60 seconds before checking again
    batch_status = openai.batches.retrieve(batch_job_id)

print("Batch job completed.")

result_file_id = batch_job.output_file_id
result_content = client.files.content(result_file_id).content

# Save the results to a file
results_file_name = "batch_filtered_code_samples.jsonl"

with open(results_file_name, 'wb') as file:
    file.write(result_content)

# Now, read the results and create a new dataset for the next steps
filtered_results = []

with open(results_file_name, 'r') as file:
    for line in file:
        json_object = json.loads(line.strip())
        task_id = json_object['custom_id']
        response_content = json_object['response']['choices'][0]['message']['content']
        # Extract the verdict
        start = response_content.find("<verdict>") + 9
        end = response_content.find("</verdict>")
        verdict = response_content[start:end].strip()
        if verdict == 'Yes':
            # Get the original code
            index = int(task_id.split('-')[1])
            code = df_filtered.iloc[index]['content']
            filtered_results.append({'index': index, 'code': code})

# Save the filtered results to a new DataFrame
df_filtered_results = pd.DataFrame(filtered_results)

# Save this DataFrame to a CSV or JSON file for later use
df_filtered_results.to_csv('filtered_code_samples.csv', index=False)

print(f"Filtered code samples saved to 'filtered_code_samples.csv'.")

Batch job status: Batch(id='batch_66fdba8d74e081909c9ba0d5e89f42ec', completion_window='24h', created_at=1727904397, endpoint='/v1/chat/completions', input_file_id='file-Hb2ynHvSbWG7onMdFjjcioEO', object='batch', status='in_progress', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1727990797, failed_at=None, finalizing_at=None, in_progress_at=1727904398, metadata=None, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=1000))
Batch job status: Batch(id='batch_66fdba8d74e081909c9ba0d5e89f42ec', completion_window='24h', created_at=1727904397, endpoint='/v1/chat/completions', input_file_id='file-Hb2ynHvSbWG7onMdFjjcioEO', object='batch', status='in_progress', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1727990797, failed_at=None, finalizing_at=None, in_progress_at=1727904398, metadata=None, output_file_id=None, 